In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from tabulate import tabulate

In [ ]:
graph = LoadConnectome2()
graph_local = LoadConnectome2("../dat/allen-mouse-conn/adj_mat_cartesian_local.csv")
graph_distal = LoadConnectome2("../dat/allen-mouse-conn/adj_mat_cartesian_distal.csv")
EdgeWeightsDict = EdgeDict(graph)
EdgeInfoDict = pk.load(open("/home/jw3514/Work/ASD_Circuits/scripts/dat/EdgeDict_selfInfo.pk", 'rb'))
str2reg, reg2str = LoadSTR2REG()
adj_mat = pd.read_csv(ConnFil, index_col=0)
adj_region_local, adj_region_distal = ConnectomeSeperation_Region(adj_mat, str2reg)
graph_region_local = LoadConnectome2(adj_region_local)
graph_region_distal = LoadConnectome2(adj_region_distal)

# Make Conn Score Matrix

In [ ]:
#OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat/"

ConnFil = "../dat/allen-mouse-conn/jw-connectome-shift.csv"
adj_mat = pd.read_csv(ConnFil, index_col=0)
OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw/"
PlainWeight_Mat = (adj_mat>0).astype(int)
PlainWeight_Mat.to_csv(OutDIR + "ScoreMat_Uniform.csv")

## Connection Density of different distance

In [ ]:
### Let's Try compute P(edge | distance percentile)

# Mask Mat2 according to values in Mat1
def MaskDistMat(Mat1, Mat2, cutoff, m='lt'):
    New_Mat2 = Mat2.copy(deep=True)
    for STR_i in Mat1.index.values:
        for STR_j in Mat1.columns.values:
            if m == 'gt':
                if Mat1.loc[STR_i, STR_j] >= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
            elif m == "lt":
                if Mat1.loc[STR_i, STR_j] <= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
    return New_Mat2
Cartesian_distances_mat = pd.read_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.csv",
                                   index_col=0)
Cartesian_distances = [x for x in Cartesian_distances_mat.values.flatten() if x > 0]
Cartesian_distances_w_edge_mat = MaskDistMat(adj_mat, Cartesian_distances_mat, cutoff=0)
Cartesian_distances_w_edge = [x for x in Cartesian_distances_w_edge_mat.values.flatten() if x > 0]
Cartesian_distances_w_edge = np.array(Cartesian_distances_w_edge)

In [ ]:
N_possible_connections, bins, xx = plt.hist(Cartesian_distances, bins=20)
N_connections, bins2, xx = plt.hist(Cartesian_distances_w_edge, bins=bins)

In [ ]:
N_possible_connections, bins

In [ ]:
Prob = []
Prob2 = []
for x, y in zip(N_connections, N_possible_connections):
    Prob.append(x/y)
    Prob2.append(1-x/y)
Info = -np.log2(np.array(Prob) )
Info2 = -np.log2(np.array(Prob2) )

In [ ]:
print(Prob)
print(Prob2)

In [ ]:
print(Info)
print(Info2)

In [ ]:
fig,ax = plt.subplots(dpi=120)
ax.plot(bins[:-1], Prob, c="black", )
ax.set_xlabel("Distance (mm)")
ax.set_ylabel("P (edge)")
ax2 = ax.twinx()
ax2.plot(bins[:-1], Info)
ax2.set_ylabel("infomation")

In [ ]:
Cartesian_distances_w_edge_mat

In [ ]:
def MakeInfoDict_Distance(bins, Info, Info2, adjMat, distMat):
    Dict = {}
    for node_i in adjMat.index.values:
        for node_j in adjMat.index.values:
            if node_i == node_j:
                continue
            else:
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        if conn > 0:
                            Dict["{}-{}".format(node_i, node_j)] = Info[i]
                        else:
                            Dict["{}-{}".format(node_i, node_j)] = Info2[i]
    return Dict

InfoDict_Distance = MakeInfoDict_Distance(bins, Info, Info2, 
                                adj_mat, Cartesian_distances_mat)
pk.dump(InfoDict, open("EdgeDict_selfInfo.Distance.v2.pk", 'wb'))
#pk.dump(PlainDict, open("dat/EdgeDict_Uniform.pk", 'wb'))

In [ ]:
def MakeInfoMat_Distance(bins, Info, Info2, adjMat, distMat):
    OutMat = pd.DataFrame(data=np.zeros((213, 213)), index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        for node_j in adjMat.index.values:
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        if conn > 0:
                            OutMat.loc[node_i, node_j] = Info[i]
                        else:
                            OutMat.loc[node_i, node_j] = Info2[i]
    return OutMat

InfoMat_Distance = MakeInfoMat_Distance(bins, Info, Info2, 
                                adj_mat, Cartesian_distances_mat)
InfoMat_Distance.to_csv("AdjMat_selfInfo.Distance.csv")
#pk.dump(InfoDict, open("EdgeDict_selfInfo.Distance.v2.csv", 'wb'))

In [ ]:
InfoMat_Distance

## Connection Density of different Region

In [ ]:
Regions = list(reg2str.keys())
Regional_Info_Dict = {}
RegionConnDen = pd.DataFrame(data=np.zeros((13,13)), index=Regions, columns=Regions)
for region_1 in Regions:
    src_strs = reg2str[region_1]
    for region_2 in Regions:
        tgt_strs = reg2str[region_2]
        sub_mat = adj_mat.loc[src_strs, tgt_strs]
        N_Possible_Conn =  sub_mat.shape[0] * sub_mat.shape[1]
        N_filled_Conn = np.count_nonzero(sub_mat)
        #print(region_1, region_2, N_filled_Conn/N_Possible_Conn)
        RegionConnDen.loc[region_1, region_2] = N_filled_Conn/N_Possible_Conn

In [ ]:
RegionConnDen

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
plt.figure(figsize=(12,12))
ax = sns.heatmap(RegionConnDen, annot=True, cmap="YlGnBu")

In [ ]:
def MakeInfoDict_region(str2reg, RegionConnDen, Mat):
    Dict = {}
    for node_i in Mat.index.values:
        reg_i = str2reg[node_i]
        for node_j in Mat.index.values:
            reg_j = str2reg[node_j]
            Prob = RegionConnDen.loc[reg_i, reg_j]
            if Prob != 0:
                Info = -np.log2(Prob)
            else:
                Info = 0
            Dict["{}-{}".format(node_i, node_j)] = Info
    return Dict
#InfoDict_Region = MakeInfoDict_region(str2reg, RegionConnDen, adj_mat)
#pk.dump(InfoDict_Region, open("EdgeDict_selfInfo.Region.pk", 'wb'))

def MakeInfoMat_Region(bins, Info, adjMat, str2reg):
    OutMat = pd.DataFrame(data=np.zeros((213, 213)), index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        reg_i = str2reg[node_i]
        for node_j in adjMat.index.values:
            reg_j = str2reg[node_j]
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                Prob = RegionConnDen.loc[reg_i, reg_j]
                conn = adjMat.loc[node_i, node_j]
                if conn > 0:
                    OutMat.loc[node_i, node_j] = -np.log2(Prob)
    return OutMat

InfoMat_Distance_V2 = MakeInfoMat_Region(bins, Infos, 
                                adj_mat, str2reg)
InfoMat_Distance_V2.to_csv(OutDIR + "ScoreMat_Region.csv")

In [ ]:
def ScoringCircuit(STRs, InfoDict):
    score = 0
    for str_i in STRs:
        for str_j in STRs:
            try:
                score += InfoDict["{}-{}".format(str_i, str_j)]
            except:
                score += 0
    return score

# Sum of total likelihood
def ScoringCircuit_v2(STRs, InfoDict):
    score = []
    for str_i in STRs:
        for str_j in STRs:
            try:
                score.append(InfoDict["{}-{}".format(str_i, str_j)])
            except:
                score.append(0)
    return np.sum(score)

# In vs Out total likehood
def ScoringCircuit_v3(STRs, InfoMat):
    Inside = 0
    Outside = 0
    ALL_STRs = InfoMat.index.values
    Out_STRs = [_str for _str in ALL_STRs if _str not in STRs]
    subMat_cir = InfoMat.loc[STRs, STRs]
    subMat_cir_out_p1 = InfoMat.loc[STRs, Out_STRs]
    subMat_cir_out_p2 = InfoMat.loc[Out_STRs, STRs]
    Inside = np.sum(subMat_cir.values)
    Outside = np.sum(subMat_cir_out_p1.values) + np.sum(subMat_cir_out_p2.values)
    return Inside, Inside - Outside

In [ ]:
%%time

topN = 50

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_score1, asd_score2 = ScoringCircuit_v3(STR_Ranks[:topN], InfoMat_Distance)


Ncont = 1000
cont_score1 = []
cont_score2 = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    score1, score2 = ScoringCircuit_v3(top_strs, InfoMat_Distance)
    cont_score1.append(score1)
    cont_score2.append(score2)
    #cont_score_reg.append(ScoringCircuit_v2(top_strs, InfoDict_Region))
    #cont_score_dist.append(ScoringCircuit_v2(top_strs, InfoDict_Distance))
    if i >= Ncont-1:
        break
cont_score1 = np.array(cont_score1)
cont_score2 = np.array(cont_score2)

In [ ]:
size = topN
#idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(cont_score1, asd_score1, ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(cont_score2, asd_score2, ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
topNs = np.arange(200, 5, -5)

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_topN_cohe_reg = []
asd_topN_cohe_dist = []
for topN in topNs:
    top_strs = STR_Ranks[:topN]
    asd_score1, asd_score2 = ScoringCircuit_v3(top_strs, InfoMat_Distance)
    asd_topN_cohe_reg.append(asd_score1)
    asd_topN_cohe_dist.append(asd_score2)
    #asd_topN_cohe_reg.append(ScoringCircuit(top_strs, InfoDict_Region))
    #asd_topN_cohe_dist.append(ScoringCircuit(top_strs, InfoDict_Distance))
asd_topN_cohe_reg = np.array(asd_topN_cohe_reg)
asd_topN_cohe_dist = np.array(asd_topN_cohe_dist)

cont_dir = "../dat/Unionize_bias/SubSampleSib/"
#cont_dir = "../dat/Unionize_bias/ASD_Sim/"
ASD_Cont_Cohesivness_Reg = []
ASD_Cont_Cohesivness_Dist = []
Ncont = 1000
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    topN_cohe_reg = []
    topN_cohe_dist = []
    for topN in topNs:
        top_strs = df.index.values[:topN]
        score1, score2 = ScoringCircuit_v3(top_strs, InfoMat_Distance)
        topN_cohe_reg.append(score1)
        topN_cohe_dist.append(score2)
        #topN_cohe_reg.append(ScoringCircuit(top_strs, InfoDict_Region))
        #topN_cohe_dist.append(ScoringCircuit(top_strs, InfoDict_Distance))
    topN_cohe_reg = np.array(topN_cohe_reg)
    topN_cohe_dist = np.array(topN_cohe_dist)
    ASD_Cont_Cohesivness_Reg.append(topN_cohe_reg)
    ASD_Cont_Cohesivness_Dist.append(topN_cohe_dist)
    if i >= Ncont-1:
        break
ASD_Cont_Cohesivness_Reg = np.array(ASD_Cont_Cohesivness_Reg)
ASD_Cont_Cohesivness_Dist = np.array(ASD_Cont_Cohesivness_Dist)

In [ ]:
def clean_inf(arr):
    print(arr.shape)
    new_arr = []
    for _arr in arr:
        FLAG=False
        for n in _arr:
            if n == 0:
                FLAG=True
                break
        if FLAG == False:
            new_arr.append(_arr)
    new_arr = np.array(new_arr)
    print(new_arr.shape)
    return new_arr

In [ ]:
fig, ax = plt.subplots(dpi=480)

#xx = asd_topN_cohe_distal / ASD_Cont_Cohesivness_Distal
#ax.errorbar(topNs-1, xx.mean(axis=0), color="lightblue", marker="o",
#            yerr=xx.std(axis=0), ls="dashed", label="ASD to ASD sim")
#xx = (asd_topN_cohe_reg - ASD_Cont_Cohesivness_Reg)/np.std(ASD_Cont_Cohesivness_Reg)
xx = (asd_topN_cohe_reg / ASD_Cont_Cohesivness_Reg)
mean = xx.mean(axis=0)
lower = np.percentile(xx, 25, axis=0)
upper = np.percentile(xx, 75, axis=0)
ax.errorbar(topNs-0.5, mean, color="blue", marker="o", markersize=3,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="Sum-likelihood")

#xx = (asd_topN_cohe_dist - clean_inf(ASD_Cont_Cohesivness_Dist))/np.std(clean_inf(ASD_Cont_Cohesivness_Dist))
xx = (asd_topN_cohe_dist / clean_inf(ASD_Cont_Cohesivness_Dist))
mean = xx.mean(axis=0)
lower = np.percentile(xx, 25, axis=0)
upper = np.percentile(xx, 75, axis=0)
ax.errorbar(topNs+0.5, mean, color="red", marker="o", markersize=3,
            yerr=(mean - lower, upper - mean ), ls="dashed", label="In/Out")

ax.grid(True)
#ax.hlines(xmin=10, xmax=200, y=1, color="grey", ls="dashed")
ax.set_xlabel("top N sturctures (by expression bias)")
ax.set_ylabel("Score ratio")
#ax.set_ylim((0.7, 3.0))
#ax.set_xlim((0, 100))
plt.legend()


In [ ]:
fig, ax = plt.subplots(dpi=480)

#xx = asd_topN_cohe_distal / ASD_Cont_Cohesivness_Distal
#ax.errorbar(topNs-1, xx.mean(axis=0), color="lightblue", marker="o",
#            yerr=xx.std(axis=0), ls="dashed", label="ASD to ASD sim")
#xx = (asd_topN_cohe_reg - ASD_Cont_Cohesivness_Reg)/np.std(ASD_Cont_Cohesivness_Reg)
xx = asd_topN_cohe_reg 
mean = xx.mean(axis=0)
#lower = np.percentile(xx, 25, axis=0)
#upper = np.percentile(xx, 75, axis=0)
#ax.errorbar(topNs-0.5, mean, color="blue", marker="o", markersize=3,
#            yerr=(mean - lower, upper - mean ), ls="dashed", label="Sum-likelihood")

#xx = (asd_topN_cohe_dist - clean_inf(ASD_Cont_Cohesivness_Dist))/np.std(clean_inf(ASD_Cont_Cohesivness_Dist))
xx = asd_topN_cohe_dist
mean = xx.mean(axis=0)
#lower = np.percentile(xx, 25, axis=0)
#upper = np.percentile(xx, 75, axis=0)
#ax.errorbar(topNs+0.5, mean, color="red", marker="o", markersize=3,
#            yerr=(mean - lower, upper - mean ), ls="dashed", label="In/Out")
ax.plot(topNs+0.5, xx, color="red", marker="o", markersize=3
            , ls="dashed", label="In/Out")

ax.plot(topNs+0.5, ASD_Cont_Cohesivness_Dist.mean(axis=0), color="blue", marker="o", markersize=3
            , ls="dashed", label="In/Out")

ax.grid(True)
#ax.hlines(xmin=10, xmax=200, y=1, color="grey", ls="dashed")
ax.set_xlabel("top N sturctures (by expression bias)")
ax.set_ylabel("Cohesivness ratio")
#ax.set_ylim((0.7, 3.0))
#ax.set_xlim((0, 100))
plt.legend()


## Expression Correlation / Specificity Correlation

In [ ]:
def MakeCorrelationMat(ExpMat):
    STRs = ExpMat.columns.values
    CorrelationMat = pd.DataFrame(data=np.zeros((213,213)), index=STRs, columns=STRs)
    for src_str in STRs: 
        for tgt_str in STRs:
            bias_src_all = ExpMat.loc[:, src_str]
            bias_tgt_all = ExpMat.loc[:, tgt_str]
            bias_correlation, p = spearmanr(bias_src_all, bias_tgt_all, 
                                                   nan_policy="omit")
            CorrelationMat.loc[src_str, tgt_str] = bias_correlation
    return CorrelationMat
ExpLevel = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/JC_ExpMat.ArithmeticMean.0418.csv", 
                      index_col=0)
ExpBias = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", 
                      index_col=0)
ExpCorrMat = MakeCorrelationMat(ExpLevel)
BiasCorrMat = MakeCorrelationMat(ExpBias)
ExpCorrMat.to_csv("ExpCorrMat.csv")
BiasCorrMat.to_csv("BiasCorrMat.csv")

In [ ]:
BiasCorrMat2

In [ ]:
adj_mat

In [ ]:
### Let's Try compute P(edge | distance percentile)

# Mask Mat2 according to values in Mat1
def MaskDistMat(Mat1, Mat2, cutoff, m='lt'):
    New_Mat2 = Mat2.copy(deep=True)
    for STR_i in Mat1.index.values:
        for STR_j in Mat1.columns.values:
            if m == 'gt':
                if Mat1.loc[STR_i, STR_j] >= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
            elif m == "lt":
                if Mat1.loc[STR_i, STR_j] <= cutoff:
                    New_Mat2.loc[STR_i, STR_j] = 0
                else:
                    New_Mat2.loc[STR_i, STR_j] = Mat2.loc[STR_i, STR_j]
    return New_Mat2
Cartesian_distances_mat = pd.read_csv("BiasCorrMat.csv",
                                   index_col=0)
Cartesian_distances = [x for x in Cartesian_distances_mat.values.flatten() if x > 0]
Cartesian_distances_w_edge_mat = MaskDistMat(adj_mat, Cartesian_distances_mat, cutoff=0)
Cartesian_distances_w_edge = [x for x in Cartesian_distances_w_edge_mat.values.flatten() if x > 0]
Cartesian_distances_w_edge = np.array(Cartesian_distances_w_edge)

In [ ]:
N_possible_connections, bins, xx = plt.hist(Cartesian_distances, bins=20)
N_connections, bins2, xx = plt.hist(Cartesian_distances_w_edge, bins=bins)

In [ ]:
Prob = []
for x, y in zip(N_connections, N_possible_connections):
    Prob.append(x/y)
Info = -np.log2(np.array(Prob) )

In [ ]:
fig,ax = plt.subplots(dpi=120)
ax.plot(bins[:-1], Prob, c="black", )
ax.set_xlabel("Distance (mm)")
ax.set_ylabel("P (edge)")
ax2 = ax.twinx()
ax2.plot(bins[:-1], Info)
ax2.set_ylabel("infomation")

In [ ]:
def MakeInfoDict_Expression(bins, Info, Mat):
    Dict1, Dict2 = {}, {}
    for node_i in Mat.index.values:
        for node_j in Mat.index.values:
            dist = Mat.loc[node_i, node_j]
            if dist == 0:
                continue
            for i in range(len(bins)-1):
                if dist >= bins[i] and dist < bins[i+1]:
                    Dict1["{}-{}".format(node_i, node_j)] = Info[i]
                    Dict2["{}-{}".format(node_i, node_j)] = 1
    return Dict1, Dict2

InfoDict_Expression, PlainDict = MakeInfoDict_Distance(bins, Info, Cartesian_distances_w_edge_mat)
pk.dump(InfoDict, open("EdgeDict_selfInfo.Expression.pk", 'wb'))

In [ ]:
topN = 50

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_score_reg = ScoringCircuit(STR_Ranks[:topN], InfoDict_Expression)
asd_score_dis = ScoringCircuit(STR_Ranks[:topN], InfoDict_Distance)


Ncont = 1000
cont_score_reg = []
cont_score_dist = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    cont_score_reg.append(ScoringCircuit(top_strs, InfoDict_Expression))
    cont_score_dist.append(ScoringCircuit(top_strs, InfoDict_Distance))
    if i >= Ncont-1:
        break
cont_score_reg = np.array(cont_score_reg)
cont_score_dist = np.array(cont_score_dist)

In [ ]:
size = topN
#idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(cont_score_reg, asd_score_reg, ax1,
                     title="ASD vs {} (Region)".format("Cont"), xlabel="cohesiveness", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(cont_score_dist, asd_score_dis, ax2,
                     title="ASD vs {} (Distal)".format("Cont"), xlabel="cohesiveness", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
topNs = np.arange(200, 5, -5)

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_topN_cohe_reg = []
asd_topN_cohe_dist = []
for topN in topNs:
    top_strs = STR_Ranks[:topN]
    asd_topN_cohe_reg.append(ScoringCircuit(top_strs, InfoDict_Region))
    asd_topN_cohe_dist.append(ScoringCircuit(top_strs, InfoDict_Distance))
asd_topN_cohe_reg = np.array(asd_topN_cohe_reg)
asd_topN_cohe_dist = np.array(asd_topN_cohe_dist)

cont_dir = "../dat/Unionize_bias/SubSampleSib/"
#cont_dir = "../dat/Unionize_bias/ASD_Sim/"
ASD_Cont_Cohesivness_Reg = []
ASD_Cont_Cohesivness_Dist = []
Ncont = 1000
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    topN_cohe_reg = []
    topN_cohe_dist = []
    for topN in topNs:
        top_strs = df.index.values[:topN]
        topN_cohe_reg.append(ScoringCircuit(top_strs, InfoDict_Region))
        topN_cohe_dist.append(ScoringCircuit(top_strs, InfoDict_Distance))
    topN_cohe_reg = np.array(topN_cohe_reg)
    topN_cohe_dist = np.array(topN_cohe_dist)
    ASD_Cont_Cohesivness_Reg.append(topN_cohe_reg)
    ASD_Cont_Cohesivness_Dist.append(topN_cohe_dist)
    if i >= Ncont-1:
        break
ASD_Cont_Cohesivness_Reg = np.array(ASD_Cont_Cohesivness_Reg)
ASD_Cont_Cohesivness_Dist = np.array(ASD_Cont_Cohesivness_Dist)

## Distance by percentile

In [ ]:
# get distance diciles
Carteisn_Dist_Mat_All = pd.read_csv("../dat/allen-mouse-conn/Dist_CartesianDistance.csv", index_col=0)
All_distances = Carteisn_Dist_Mat_All.values.flatten()
DistanceDeciles = np.percentile(All_distances, np.arange(0, 100, 10))
DistanceDeciles = np.append(DistanceDeciles, max(All_distances))

In [ ]:
distances = Carteisn_Dist_Mat_All[adj_mat>0]
distances = distances[~np.isnan(distances)]

In [ ]:
Counts_all, bins = np.histogram(All_distances, bins=DistanceDeciles)
Counts_conn, bins = np.histogram(distances, bins=DistanceDeciles)
r = range(len(bins)-1)
names = ["%.0f-%.0f"%(bins[i], bins[i+1]) for i in range(len(bins)-1)]
barWidth = 0.85
plt.bar(r, Counts_all, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
plt.bar(r, Counts_conn, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
plt.xticks(r, names)
plt.xlabel("group")
plt.xticks(rotation = 45) 
plt.show()

In [ ]:
Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

In [ ]:
fig,ax = plt.subplots(dpi=120)
ax.plot(bins[:-1], Probs, c="black", marker='o')
ax.set_xlabel("Distance (mm)")
ax.set_ylabel("P (edge)")
ax2 = ax.twinx()
ax2.plot(bins[:-1], Infos, marker='o')
ax2.set_ylabel("infomation")

In [ ]:
def MakeInfoMat_Distance(bins, Info, adjMat, distMat):
    OutMat = pd.DataFrame(data=np.zeros((213, 213)), index=adjMat.index.values, columns=adjMat.columns.values)
    OutMat2 = pd.DataFrame(data=np.zeros((213, 213)), index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        for node_j in adjMat.index.values:
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        OutMat2.loc[node_i, node_j] = Info[i] #np.exp(-Info[i])
                        if conn > 0:
                            OutMat.loc[node_i, node_j] = Info[i]
    return OutMat, OutMat2

OutDIR = "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/ScoreingMat_jw/"
InfoMat_Distance_V2, XX = MakeInfoMat_Distance(bins, Infos, 
                                adj_mat, Carteisn_Dist_Mat_All)
InfoMat_Distance_V2.to_csv(OutDIR + "ScoreMat_Distance.csv")
XX.to_csv(OutDIR + "ScoreMat_Distance.Entropy.csv")

In [ ]:
adj_short, adj_long = ConnectomeSeperation_Distance(InfoMat_Distance_V2, Carteisn_Dist_Mat_All)
adj_short.to_csv(OutDIR + "ScoreMat_Distance.short.csv")
adj_long.to_csv(OutDIR + "ScoreMat_Distance.long.csv")

In [ ]:
print(np.count_nonzero(InfoMat_Distance_V2))
print(np.count_nonzero(adj_short))
print(np.count_nonzero(adj_long))

In [ ]:
Cartesian_distances_w_edge = MaskDistMat(adj_mat, dist_mat, cutoff=0)
mid = np.median([x for x in Cartesian_distances_w_edge.values.flatten() if x > 0])
adj_mat_short = pd.DataFrame(data=np.zeros((213,213)),index=adj_mat.index.values,columns=adj_mat.columns.values)
adj_mat_long = pd.DataFrame(data=np.zeros((213,213)),index=adj_mat.index.values,columns=adj_mat.columns.values)

for str_i in adj_mat.index.values:
    for str_i in adj_mat.index.values:
        conn_w = adj_mat.loc[str_i, str_j]
        conn_d = dist_mat.loc[str_i, str_j]
        if conn

In [ ]:
adj_short, adj_long = ConnectomeSeperation_Distance(XX, Carteisn_Dist_Mat_All)
adj_short.to_csv(OutDIR + "ScoreMat_Distance.short.csv")
adj_long.to_csv(OutDIR + "ScoreMat_Distance.long.csv")
print(np.count_nonzero(XX))
print(np.count_nonzero(adj_short))
print(np.count_nonzero(adj_long))

In [ ]:
def ScoringCircuit_v3(STRs, InfoMat):
    Inside = 0
    Outside = 0
    ALL_STRs = InfoMat.index.values
    Out_STRs = [_str for _str in ALL_STRs if _str not in STRs]
    subMat_cir = InfoMat.loc[STRs, STRs]
    subMat_cir_out_p1 = InfoMat.loc[STRs, Out_STRs]
    subMat_cir_out_p2 = InfoMat.loc[Out_STRs, STRs]
    Inside = np.sum(subMat_cir.values)
    Outside = np.sum(subMat_cir_out_p1.values) + np.sum(subMat_cir_out_p2.values)
    return Inside, Inside - Outside

In [ ]:
%%time

topN = 50

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_score1, asd_score2 = ScoringCircuit_v3(STR_Ranks[:topN], InfoMat_Distance_V2)


Ncont = 1000
cont_score1 = []
cont_score2 = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    score1, score2 = ScoringCircuit_v3(top_strs, InfoMat_Distance_V2)
    cont_score1.append(score1)
    cont_score2.append(score2)
    #cont_score_reg.append(ScoringCircuit_v2(top_strs, InfoDict_Region))
    #cont_score_dist.append(ScoringCircuit_v2(top_strs, InfoDict_Distance))
    if i >= Ncont-1:
        break
cont_score1 = np.array(cont_score1)
cont_score2 = np.array(cont_score2)

In [ ]:
size = topN
#idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(cont_score1, asd_score1, ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(cont_score2, asd_score2, ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
def get_score_dist(STRs, scoreMat):
    res = scoreMat.loc[STRs, STRs].values
    res = res[~np.isnan(res)]
    return res

In [ ]:
EdgeMat = Carteisn_Dist_Mat_All[adj_mat>0]
ASD_dist = get_score_dist(STR_Ranks[:topN], EdgeMat)

In [ ]:
Ncont = 1000
cont_score_dist = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    cont_dist = get_score_dist(top_strs, EdgeMat)
    cont_score_dist.extend(list(cont_dist))
    if i >= Ncont-1:
        break
cont_score_dist = np.array(cont_score_dist)

In [ ]:
n, bins, patches = plt.hist(ASD_dist, color="red", density=1, alpha=0.5)
plt.hist(cont_score_dist, bins=bins, color="blue", density=1, alpha=0.5)

## Correlation distance

In [ ]:
ExpCorrMat = pd.read_csv("ExpCorrMat.csv", index_col=0)
BiasCorrMat = pd.read_csv("BiasCorrMat.csv", index_col=0)

In [ ]:
All_correlation = BiasCorrMat.values.flatten()
correlationDeciles = np.percentile(All_correlation, np.arange(0, 100, 10))
correlationDeciles = np.append(correlationDeciles, max(All_correlation))

In [ ]:
correlationDeciles

In [ ]:
corrlations = BiasCorrMat[adj_mat>0]
corrlations = corrlations[~np.isnan(corrlations)]

In [ ]:
Counts_all, bins = np.histogram(All_correlation, bins=correlationDeciles)
Counts_conn, bins = np.histogram(corrlations, bins=correlationDeciles)
r = range(len(bins)-1)
names = ["%.2f-%.2f"%(bins[i], bins[i+1]) for i in range(len(bins)-1)]
barWidth = 0.85
plt.bar(r, Counts_all, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
plt.bar(r, Counts_conn, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
plt.xticks(r, names)
plt.xlabel("group")
plt.xticks(rotation = 45) 
plt.show()

In [ ]:
Probs = Counts_conn/Counts_all
Infos = -np.log2(Probs)

In [ ]:
fig,ax = plt.subplots(dpi=120)
ax.plot(bins[:-1], Probs, c="black", marker='o')
ax.set_xlabel("Distance (mm)")
ax.set_ylabel("P (edge)")
ax2 = ax.twinx()
ax2.plot(bins[:-1], Infos, marker='o')
ax2.set_ylabel("infomation")

In [ ]:
def MakeInfoMat_Distance(bins, Info, adjMat, distMat):
    OutMat = pd.DataFrame(data=np.zeros((213, 213)), index=adjMat.index.values, columns=adjMat.columns.values)
    for node_i in adjMat.index.values:
        for node_j in adjMat.index.values:
            if node_i == node_j:
                OutMat.loc[node_i, node_j] = 0
            else:
                dist = distMat.loc[node_i, node_j]
                conn = adjMat.loc[node_i, node_j]
                for i in range(len(bins)-1):
                    if dist >= bins[i] and dist < bins[i+1]:
                        if conn > 0:
                            OutMat.loc[node_i, node_j] = Info[i]
    return OutMat

InfoMat_BiasCorr = MakeInfoMat_Distance(bins, Infos, 
                                adj_mat, BiasCorrMat)
InfoMat_BiasCorr.to_csv(OutDIR + "ScoreMat_BiasCorr.csv")

In [ ]:
topN = 50

STR_Ranks = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR").index.values
asd_score1, asd_score2 = ScoringCircuit_v3(STR_Ranks[:topN], InfoMat_BiasCorr)


Ncont = 1000
cont_score1 = []
cont_score2 = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    score1, score2 = ScoringCircuit_v3(top_strs, InfoMat_BiasCorr)
    cont_score1.append(score1)
    cont_score2.append(score2)
    #cont_score_reg.append(ScoringCircuit_v2(top_strs, InfoDict_Region))
    #cont_score_dist.append(ScoringCircuit_v2(top_strs, InfoDict_Distance))
    if i >= Ncont-1:
        break
cont_score1 = np.array(cont_score1)
cont_score2 = np.array(cont_score2)

In [ ]:
size = topN
#idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(cont_score1, asd_score1, ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(cont_score2, asd_score2, ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
EdgeMat = BiasCorrMat[adj_mat>0]
ASD_dist = get_score_dist(STR_Ranks[:topN], EdgeMat)
Ncont = 1000
cont_score_dist = []
cont_dir = "../dat/Unionize_bias/SubSampleSib/"
for i, file in enumerate(os.listdir(cont_dir)):
    if file.startswith("cont.genes"):
        continue
    df = pd.read_csv(cont_dir + file, index_col="STR")
    top_strs = df.index.values[:topN]
    cont_dist = get_score_dist(top_strs, EdgeMat)
    cont_score_dist.extend(list(cont_dist))
    if i >= Ncont-1:
        break
cont_score_dist = np.array(cont_score_dist)

In [ ]:
n, bins, patches = plt.hist(ASD_dist, color="red", density=1, alpha=0.5)
plt.hist(cont_score_dist, bins=bins, color="blue", density=1, alpha=0.5)

In [ ]:
plt.scatter(corrlations, distances, s=0.1)

# Test 

In [ ]:
ScoreMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw/ScoreMat_Distance.csv", index_col=0)
ScoreMat2 = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw/ScoreMat_Distance.Entropy.csv", index_col=0)
#ScoreMat2 = np.exp2(-ScoreMat2)
#ScoreMat2[ScoreMat2==1] = 0
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")
test_ns = np.arange(20, 200, 10)

In [ ]:
STR_Ranks = ASD_BiasDF.index.values
asd_topN_cohe_1, asd_topN_cohe_2 = [], []
for topN in test_ns:
    top_strs = STR_Ranks[:topN]
    #asd_score1, asd_score2 = ScoringCircuit_v4(top_strs, ScoreMat, ScoreMat2)
    asd_score1, asd_score2 = ScoringCircuit_v5(top_strs, ScoreMat, ScoreMat2)
    asd_topN_cohe_1.append(asd_score1)
    asd_topN_cohe_2.append(asd_score2)
asd_topN_cohe_1 = np.array(asd_topN_cohe_1)
asd_topN_cohe_2 = np.array(asd_topN_cohe_2)

In [ ]:
print(asd_topN_cohe_1.shape, asd_topN_cohe_2.shape)

In [ ]:
asd_topN_cohe_2

In [ ]:
plt.plot(test_ns, asd_topN_cohe_1, color="red")
plt.plot(test_ns, asd_topN_cohe_2, color="blue")

# Scoring

In [ ]:
def PlotingScore_profile(ASD, Cont, title="", method = "ratio"):
    fig, ax = plt.subplots(dpi=480)
    if method == "ratio":
        xx = ASD/Cont
        mean = xx.mean(axis=0)
        lower = np.percentile(xx, 25, axis=0)
        upper = np.percentile(xx, 75, axis=0)
        ax.errorbar(topNs, mean, color="red", marker="o", markersize=3, lw=0.1,
                    yerr=(mean - lower, upper - mean ), ls="dashed", label="ASD/Sib")
    elif method == "zscore":
        xx = (ASD - Cont.mean(axis=0))/Cont.std(axis=0)
        ax.plot(topNs, xx, color="red", marker="o", markersize=3, lw=0.1,
                     ls="dashed", label="ASD to Sib Zscore")
    elif method == "diff":
        #xx = np.exp2(ASD - Cont.mean(axis=0))
        xx = ASD - Cont.mean(axis=0)
        ax.plot(topNs, xx, color="blue", marker="o", markersize=3, lw=0.1,
                     ls="dashed", label="ASD to Sib Zscore")
        #ax.plot(topNs, xx/((topNs)*topNs), color="red", marker="o", markersize=3, lw=0.1,
        #             ls="dashed", label="ASD to Sib Zscore")
    ax.grid(True)
    ax.set_title(title)
    ax.set_xlabel("top N sturctures (by expression bias)")
    ax.set_ylabel("Score ratio")
    plt.legend()
    #ax.set_xlim(1,100)


In [ ]:
topNs = np.arange(200, 5, -1)
ASD_Distance_In = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.Distance.ASD.In.npy")
Cont_Distance_In = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.Distance.Cont.In.npy")

ASD_Distance_InOut = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.Distance.ASD.InOut.npy")
Cont_Distance_InOut = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.Distance.Cont.InOut.npy")

In [ ]:
PlotingScore_profile(ASD_Distance_In, Cont_Distance_In, "Distance In", method="diff")

In [ ]:
PlotingScore_profile(ASD_Distance_In, Cont_Distance_In, "Distance In", method="diff")

In [ ]:
idx = np.where(topNs==50)[0][0]
(ASD_Distance_In - Cont_Distance_In.mean(axis=0))[idx]

In [ ]:
250 / (50*50)

In [ ]:
750 / (90*90)

In [ ]:
PlotingScore_profile(ASD_Distance_In, Cont_Distance_In, "Distance In", method="diff")

In [ ]:
PlotingScore_profile(ASD_Distance_In, Cont_Distance_In, "Distance In")
PlotingScore_profile(ASD_Distance_InOut, Cont_Distance_InOut, "Distance InOut")
PlotingScore_profile(ASD_Distance_In, Cont_Distance_In, "Distance In", method="zscore")
PlotingScore_profile(ASD_Distance_InOut, Cont_Distance_InOut, "Distance InOut", method="zscore")
PlotingScore_profile(np.exp2(ASD_Distance_In), Cont_Distance_In, "Distance In", method="diff")
PlotingScore_profile(ASD_Distance_InOut, Cont_Distance_InOut, "Distance InOut", method="diff")

#### test ASD stability 

In [ ]:
def BlockShuffle(List, BlockSize):
    new_list = []
    for i in range(0, len(List), BlockSize):
        if i + BlockSize < len(List):
            x = list(List[i:i+BlockSize])
            x = random.sample(x, BlockSize)
            new_list.extend(x)
        else:
            x = list(List[i: len(List)])
            x = random.sample(x, len(x))     
            new_list.extend(x)
    return new_list

def run_profile(Ranks, ScoreMat, topNs=topNs):
    scores = []
    for topN in topNs:
        top_strs = Ranks[:topN]
        asd_score1, asd_score2 = ScoringCircuit_v3(top_strs, ScoreMat)
        scores.append(asd_score1)
    return scores

In [ ]:
ScoreMat = pd.read_csv("../dat/allen-mouse-conn/ScoreingMat_jw/ScoreMat_Distance.csv", index_col=0)
ASD_BiasDF = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", index_col="STR")
Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 10)

In [ ]:
scores0 = run_profile(ASD_BiasDF.index.values, ScoreMat)
PlotingScore_profile(scores, Cont_Distance_In, "Distance In", method="diff")

In [ ]:
Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 30)
scores = run_profile(Shuff_Ranks, ScoreMat)
PlotingScore_profile(np.exp2(scores), Cont_Distance_In, "Distance In", method="diff")

In [ ]:
idx_50 = np.where(topNs==50)[0][0]
idx_90 = np.where(topNs==90)[0][0]
idx_170 = np.where(topNs==170)[0][0]

In [ ]:
# Get P values for different site 
scores_50, socres_90, scores_175 = [], [], []
for i in range(1000):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 213)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])
    #print(xx.shape)

In [ ]:
size = 50
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2, ax3) = plt.subplots(1,3,dpi=120, figsize=(16,4))
PlotPermutationP(scores_50, 250, ax1,
                     title="50", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(socres_90, 750, ax2,
                     title="90", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(scores_175, 800, ax3,
                     title="170", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")

In [ ]:
scores_50, socres_90, scores_175 = [], [], []
for i in range(1000):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 50)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])

In [ ]:
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2, ax3) = plt.subplots(1,3,dpi=120, figsize=(16,4))
PlotPermutationP(scores_50, 250, ax1,
                     title="50", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(socres_90, 750, ax2,
                     title="90", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(scores_175, 800, ax3,
                     title="170", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")

In [ ]:
scores_50, socres_90, scores_175 = [], [], []
for i in range(1000):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 60)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])

In [ ]:
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2, ax3) = plt.subplots(1,3,dpi=120, figsize=(16,4))
PlotPermutationP(scores_50, 250, ax1,
                     title="50", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(socres_90, 750, ax2,
                     title="90", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")
PlotPermutationP(scores_175, 800, ax3,
                     title="170", xlabel="Score", 
                     dist_label="Rand - Sib", bar_label="ASD - Sib")

In [ ]:
scores_50, socres_90, scores_175 = [], [], []
All_rands = []
for i in range(100):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 125)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])
    All_rands.append(xx)
All_rands = np.array(All_rands)

In [ ]:
fig, ax = plt.subplots(dpi=480)
ax.plot(topNs, All_rands.mean(axis=0), color="red", marker="o", markersize=3, lw=0.1,
                     ls="dashed", label="ASD to Sib Zscore")

In [ ]:
scores_50, socres_90, scores_175 = [], [], []
All_rands = []
for i in range(50):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 50)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])
    All_rands.append(xx)
All_rands = np.array(All_rands)
fig, ax = plt.subplots(dpi=480)
ax.plot(topNs, All_rands.mean(axis=0), color="red", marker="o", markersize=3, lw=0.1,
                     ls="dashed", label="ASD to Sib Zscore")
ax.grid(True)

In [ ]:
scores_50, socres_90, scores_175 = [], [], []
All_rands = []
for i in range(50):
    Shuff_Ranks = BlockShuffle(ASD_BiasDF.index.values, 25)
    scores = run_profile(Shuff_Ranks, ScoreMat)
    xx = scores - Cont_Distance_In.mean(axis=0)
    #print(xx)
    scores_50.append(xx[idx_50])
    socres_90.append(xx[idx_90])
    scores_175.append(xx[idx_170])
    All_rands.append(xx)
All_rands = np.array(All_rands)
fig, ax = plt.subplots(dpi=480)
ax.plot(topNs, All_rands.mean(axis=0), color="red", marker="o", markersize=3, lw=0.1,
                     ls="dashed", label="ASD to Sib Zscore")
ax.grid(True)

In [ ]:
size = 50
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(Cont_Distance_In[:, idx], ASD_Distance_In[idx], ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(Cont_Distance_InOut[:, idx], ASD_Distance_InOut[idx], ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
ASD_Distance_In_Short = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.DistanceShort.ASD.In.npy")
Cont_Distance_In_Short = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.DistanceShort.Cont.In.npy")

ASD_Distance_In_Long = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.DistanceLong.ASD.In.npy")
Cont_Distance_In_Long = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.DistanceLong.Cont.In.npy")

In [ ]:
size = 50
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(Cont_Distance_In_Short[:, idx], ASD_Distance_In_Short[idx], ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(Cont_Distance_In_Long[:, idx], ASD_Distance_In_Long[idx], ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
PlotingScore_profile(ASD_Distance_In_Short, Cont_Distance_In_Short, "Shoart Distance In")
PlotingScore_profile(ASD_Distance_In_Long, Cont_Distance_In_Long, "Long Distance In")

In [ ]:
ASD_Uniform_In = np.load("../dat/Unionize_bias/RankScore.Uniform.ASD.In.npy")
Cont_Uniform_In = np.load("../dat/Unionize_bias/RankScore.Uniform.Cont.In.npy")

ASD_Uniform_InOut = np.load("../scripts/RankScore.Uniform.ASD.InOut.npy")
Cont_Uniform_InOut = np.load("../scripts/RankScore.Uniform.Cont.InOut.npy")

In [ ]:
PlotingScore_profile(ASD_Uniform_In, Cont_Uniform_In, "Uniform In")
PlotingScore_profile(ASD_Uniform_InOut, Cont_Uniform_InOut, "Uniform InOut")

In [ ]:
#ASD_ConnWeight_In = np.load("../dat/Unionize_bias/RankScore.ConnWeight.ASD.In.npy")
#Cont_ConnWeight_In = np.load("../dat/Unionize_bias/RankScore.ConnWeight.Cont.In.npy")

#ASD_ConnWeight_InOut = np.load("../scripts/RankScore.ConnWeight.ASD.InOut.npy")
#Cont_ConnWeight_InOut = np.load("../scripts/RankScore.ConnWeight.Cont.InOut.npy")

ASD_ConnWeight_In = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.ConnWeight.ASD.In.npy")
Cont_ConnWeight_In = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.ConnWeight.Cont.In.npy")

ASD_ConnWeight_InOut = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.ConnWeight.ASD.InOut.npy")
Cont_ConnWeight_InOut = np.load("../dat/Unionize_bias/RankScore_jw/RankScore.ConnWeight.Cont.InOut.npy")

In [ ]:
PlotingScore_profile(ASD_ConnWeight_In, Cont_ConnWeight_In, "ConnWeight In")
PlotingScore_profile(ASD_ConnWeight_InOut, Cont_ConnWeight_InOut, "ConnWeight InOut")

In [ ]:
size = 50
idx = np.where(topNs==size)[0][0]
fig, (ax1, ax2) = plt.subplots(1,2,dpi=120, figsize=(8,4))
PlotPermutationP(Cont_ConnWeight_In[:, idx], ASD_ConnWeight_In[idx], ax1,
                     title="Likelihood Inside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")
PlotPermutationP(Cont_ConnWeight_InOut[:, idx], ASD_ConnWeight_InOut[idx], ax2,
                     title="Likelihood Inside / Outside Circuit".format(), xlabel="Score", 
                     dist_label="ASD Sim", bar_label="ASD")

In [ ]:
ASD_BiasCorr_In = np.load("../dat/Unionize_bias/RankScore.BiasCorr.ASD.In.npy")
Cont_BiasCorr_In = np.load("../dat/Unionize_bias/RankScore.BiasCorr.Cont.In.npy")

ASD_BiasCorr_InOut = np.load("../scripts/RankScore.BiasCorr.ASD.InOut.npy")
Cont_BiasCorr_InOut = np.load("../scripts/RankScore.BiasCorr.Cont.InOut.npy")

In [ ]:
PlotingScore_profile(ASD_BiasCorr_In, Cont_BiasCorr_In, "BiasCorr In")
PlotingScore_profile(ASD_BiasCorr_InOut, Cont_BiasCorr_InOut, "BiasCorr InOut")

In [ ]:
ASD_Region_In = np.load("../dat/Unionize_bias/RankScore.Region.ASD.In.npy")
Cont_Region_In = np.load("../dat/Unionize_bias/RankScore.Region.Cont.In.npy")

ASD_Region_InOut = np.load("../scripts/RankScore.Region.ASD.InOut.npy")
Cont_Region_InOut = np.load("../scripts/RankScore.Region.Cont.InOut.npy")

In [ ]:
PlotingScore_profile(ASD_Region_In, Cont_Region_In, "Region In")
PlotingScore_profile(ASD_Region_InOut, Cont_Region_InOut, "Region InOut")

# Compare Scores

In [ ]:
DIR = "../dat/allen-mouse-conn/ScoreingMat/"

BiasCorr = pd.read_csv(DIR + "ScoreMat_BiasCorr.csv", index_col=0)
ConnWeight = pd.read_csv(DIR + "ScoreMat_ConnWeights.csv", index_col=0)
#ConnWeight = pd.read_csv( "/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight_pvalue.csv", index_col=0)
Distance = pd.read_csv(DIR + "ScoreMat_Distance.csv", index_col=0)
Region = pd.read_csv(DIR + "ScoreMat_Region.csv", index_col=0)

In [ ]:
ScoreDF_dat = []
for i_str in BiasCorr.index.values:
    for j_str in BiasCorr.columns.values:
        a = BiasCorr.loc[i_str, j_str]
        b = ConnWeight.loc[i_str, j_str]
        c = Distance.loc[i_str, j_str]
        d = Region.loc[i_str, j_str]
        if a != 0:
            name = "{}-{}".format(i_str, j_str)
            ScoreDF_dat.append([name, a,b,c,d])
ScoreDF = pd.DataFrame(ScoreDF_dat, columns=["Edge", "BiasCorr", "ConnWeight", "Distance", "Region"])
ScoreDF = ScoreDF.set_index("Edge")


In [ ]:
import seaborn as sns
sns.pairplot(ScoreDF)

In [ ]:
ASD_STRs = pd.read_csv("../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", 
                       index_col="STR").index.values[:50]

In [ ]:
ScoreDF_dat_asd = []
for i_str in BiasCorr.index.values:
    for j_str in BiasCorr.columns.values:
        if (i_str in ASD_STRs and j_str in ASD_STRs):
            a = BiasCorr.loc[i_str, j_str]
            b = ConnWeight.loc[i_str, j_str]
            c = Distance.loc[i_str, j_str]
            d = Region.loc[i_str, j_str]
            if a != 0:
                name = "{}-{}".format(i_str, j_str)
                ScoreDF_dat_asd.append([name, a,b,c,d])
ASD_ScoreDF = pd.DataFrame(ScoreDF_dat_asd, columns=["Edge", "BiasCorr", "ConnWeight", "Distance", "Region"])
ASD_ScoreDF = ASD_ScoreDF.set_index("Edge")

In [ ]:
ASD_ScoreDF

In [ ]:
import seaborn as sns
sns.pairplot(ASD_ScoreDF)